In [2]:
import web3 as w3

In [3]:
from solcx import compile_solc

compiled_solidity = compile_solc(
    '''
// SPDX-License-Identifier: MIT
// OpenZeppelin Contracts (last updated v4.6.0) (finance/VestingWallet.sol)
pragma solidity ^0.8.0;

import "https://github.com/OpenZeppelin/openzeppelin-contracts/blob/master/contracts/token/ERC20/utils/SafeERC20.sol";
import "https://github.com/OpenZeppelin/openzeppelin-contracts/blob/master/contracts/utils/Address.sol";
import "https://github.com/OpenZeppelin/openzeppelin-contracts/blob/master/contracts/utils/Context.sol";
import "https://github.com/OpenZeppelin/openzeppelin-contracts/blob/master/contracts/utils/math/Math.sol";

/**
 * @title VestingWallet
 * @dev This contract handles the vesting of Eth and ERC20 tokens for a given beneficiary. Custody of multiple tokens
 * can be given to this contract, which will release the token to the beneficiary following a given vesting schedule.
 * The vesting schedule is customizable through the {vestedAmount} function.
 *
 * Any token transferred to this contract will follow the vesting schedule as if they were locked from the beginning.
 * Consequently, if the vesting has already started, any amount of tokens sent to this contract will (at least partly)
 * be immediately releasable.
 */
contract VestingWallet is Context {
    event EtherReleased(uint256 amount);
    event ERC20Released(address indexed token, uint256 amount);

    uint256 private _released;
    mapping(address => uint256) private _erc20Released;
    address private immutable _beneficiary;
    uint64 private immutable _start;
    uint64 private immutable _duration;

    /**
     * @dev Set the beneficiary, start timestamp and vesting duration of the vesting wallet.
     */
    constructor(
        address beneficiaryAddress,
        uint64 startTimestamp,
        uint64 durationSeconds
    ) public {
        require(beneficiaryAddress != address(0), "VestingWallet: beneficiary is zero address");
         _beneficiary = beneficiaryAddress;
         _start = startTimestamp;
         _duration = durationSeconds;
    }

    /**
     * @dev The contract should be able to receive Eth.
     */
    receive() external payable virtual {}

    /**
     * @dev Getter for the beneficiary address.
     */
    function beneficiary() public view virtual returns (address) {
        return _beneficiary;
    }

    /**
     * @dev Getter for the start timestamp.
     */
    function start() public view virtual returns (uint256) {
        return _start;
    }

    /**
     * @dev Getter for the vesting duration.
     */
    function duration() public view virtual returns (uint256) {
        return _duration;
    }

    /**
     * @dev Amount of eth already released
     */
    function released() public view virtual returns (uint256) {
        return _released;
    }

    /**
     * @dev Amount of token already released
     */
    function released(address token) public view virtual returns (uint256) {
        return _erc20Released[token];
    }

    /**
     * @dev Release the native token (ether) that have already vested.
     *
     * Emits a {EtherReleased} event.
     */
    function release() public virtual {
        uint256 releasable = vestedAmount(uint64(block.timestamp)) - released();
        _released += releasable;
        emit EtherReleased(releasable);
        Address.sendValue(payable(beneficiary()), releasable);
    }

    /**
     * @dev Release the tokens that have already vested.
     *
     * Emits a {ERC20Released} event.
     */
    function release(address token) public virtual {
        uint256 releasable = vestedAmount(token, uint64(block.timestamp)) - released(token);
        _erc20Released[token] += releasable;
        emit ERC20Released(token, releasable);
        SafeERC20.safeTransfer(IERC20(token), beneficiary(), releasable);
    }

    /**
     * @dev Calculates the amount of ether that has already vested. Default implementation is a linear vesting curve.
     */
    function vestedAmount(uint64 timestamp) public view virtual returns (uint256) {
        return _vestingSchedule(address(this).balance + released(), timestamp);
    }

    /**
     * @dev Calculates the amount of tokens that has already vested. Default implementation is a linear vesting curve.
     */
    function vestedAmount(address token, uint64 timestamp) public view virtual returns (uint256) {
        return _vestingSchedule(IERC20(token).balanceOf(address(this)) + released(token), timestamp);
    }

    /**
     * @dev Virtual implementation of the vesting formula. This returns the amount vested, as a function of time, for
     * an asset given its total historical allocation.
     */
    function _vestingSchedule(uint256 totalAllocation, uint64 timestamp) internal view virtual returns (uint256) {
        if (timestamp < start()) {
            return 0;
        } else if (timestamp > start() + duration()) {
            return totalAllocation;
        } else {
            return (totalAllocation * (timestamp - start())) / duration();
        }
    }
}
''', output_values = ['abi', 'bin'] )

TypeError: compile_solc() got an unexpected keyword argument 'output_values'

In [5]:
compiled_solidity

NameError: name 'compiled_solidity' is not defined

In [ ]:
contract_id, contract_interface = compiled_solidity.popitem()

print(contract_id)

In [ ]:
print(contract_interface)

In [ ]:
contract_interface['abi']

In [ ]:
contrace_interface['bin']

In [3]:
# provider for node

web3=w3(w3.EthereumTesterProvider())

#prefunded account as sender

w3.eth.default_account = w3.eth.accounts[0]

abi = contract_interface[abi]

project3 = w3.eth.contract(abi = abi, bytecode=bytecode)
print(project3)

/Users/cliffordcharles/opt/anaconda3/lib/python3.9/site-packages/eth_tester/backends/__init__.py:30: UserWarning: Ethereum Tester: No backend was explicitely set, and no *full* backends were available.  Falling back to the `MockBackend` which does not support all EVM functionality.  Please refer to the `eth-tester` documentation for information on what backends are available and how to set them.
  warnings.warn(UserWarning(


TypeError: 'module' object is not callable

In [ ]:
tx_hash = project3.constructor().transact()

tx_hash

In [ ]:
tx_receipt=w3.eth.wait_for_transaction_receipt(tx_hash=tx_hash)

tx_receipt

In [ ]:
#use address to get the contract
project3Contract = w3.eth.contract(address = tx_receipt.contractAddress,
                                  abi = abi)

project3Contract 

In [ ]:
#call functions from contract

project3Contract.functions.vestedAmount().call()


In [ ]:
#modify transactions (will cost gas)
bye_hash = project3Contract.functions.vestedAmount().transact()

In [ ]:
bye_hash

In [ ]:
bye_receipt = w3.eth.wait_for_transaction_receipt(bye_hash)

In [ ]:
bye_receipt